# 模拟登录

- 爬取基于某些用户的用户信息

需求 1: 对古诗文网进行模拟登录【人人网注册不了了】

1. 点击登录之后发起 post 请求
2. post 请求中会携带登陆之前录入的相关的登陆信息【用户名、密码、验证码】
3. 验证码: 每次请求都会发生变化

需求 2: 爬取当前用户的相关的用户信息【个人主页的用户信息】

http/https 协议特性: 无状态

    登录状态下，没有请求到对应页面数据的愿意:
        发起的第二次基于个人主页请求的时候，服务器不知道此请求是基于登陆状态下的请求

cookie: 用来让服务器端记录客户端的相关状态

1. 手动处理
    - 不推荐【麻烦，需要手动在请求头中加 cookie】
2. 自动处理
    - cookie 值的来源 ?
    模拟登陆 post 请求后，由服务器端创建
    - session 会话对象
    作用: 
        1.可以进行请求的发送
        2.如果请求过程中产生了 cookie，则该 cookie 会被自动存储/携带在该 session 对象中
    
    流程:
        1.创建一个 session 对象
        2.使用 session 对象进行模拟登录 post 请求的发送【cookie 就会被存储在 session 中】
        3.session 对象对个人主页对应的 get 请求进行发送【携带了 cookie】


# 编码流程

1. 验证码识别，获取验证码图片的文字数据
2. 发起 post 请求【处理请求参数】
3. 对响应数据进行持久化存储

# 包

In [21]:
import requests
from lxml import etree
# 超级鹰用到的包
from hashlib import md5

# static data

In [22]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'
}

url = 'https://so.gushiwen.cn/user/login.aspx?from=http://so.gushiwen.cn/user/collect.aspx'


# 获取验证码图片

In [23]:

page_text = requests.get(url = url, headers = headers).text
tree = etree.HTML(page_text)
# 获取验证码图片 src 地址
src_list = tree.xpath('//*[@id="imgCode"]/@src')[0]
url_code = 'https://so.gushiwen.cn' + src_list
# 请求验证码图片
img_data = requests.get(url = url_code, headers = headers).content
# 持久化存储验证码图片
with open('./code.jpg', 'wb') as fp:
    fp.write(img_data)

# 实例化超级鹰

In [24]:
# 超级鹰封装好的一个类，待会直接 实例化即可
class Chaojiying_Client(object):

    def __init__(self, username, password, soft_id):
        self.username = username
        password =  password.encode('utf8')
        self.password = md5(password).hexdigest()
        self.soft_id = soft_id
        self.base_params = {
            'user': self.username,
            'pass2': self.password,
            'softid': self.soft_id,
        }
        self.headers = {
            'Connection': 'Keep-Alive',
            'User-Agent': 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 5.1; Trident/4.0)',
        }

    def PostPic(self, im, codetype):
        """
        im: 图片字节
        codetype: 题目类型 参考 http://www.chaojiying.com/price.html
        """
        params = {
            'codetype': codetype,
        }
        params.update(self.base_params)
        files = {'userfile': ('ccc.jpg', im)}
        r = requests.post('http://upload.chaojiying.net/Upload/Processing.php', data=params, files=files, headers=self.headers)
        return r.json()

    def PostPic_base64(self, base64_str, codetype):
        """
        im: 图片字节
        codetype: 题目类型 参考 http://www.chaojiying.com/price.html
        """
        params = {
            'codetype': codetype,
            'file_base64':base64_str
        }
        params.update(self.base_params)
        r = requests.post('http://upload.chaojiying.net/Upload/Processing.php', data=params, headers=self.headers)
        return r.json()

    def ReportError(self, im_id):
        """
        im_id:报错题目的图片ID
        """
        params = {
            'id': im_id,
        }
        params.update(self.base_params)
        r = requests.post('http://upload.chaojiying.net/Upload/ReportError.php', data=params, headers=self.headers)
        return r.json()

#用户中心>>软件ID 生成一个替换 96001
#本地图片文件路径 来替换 a.jpg 有时WIN系统须要//
#1902 验证码类型  官方网站>>价格体系 3.4+版 print 后要加()
chaojiying = Chaojiying_Client('1940335829', 'jtycbbt123', '938368')	
im = open('./code.jpg', 'rb').read()	
# chaojiying.PostPic(im, 1902) 返回一个字典，pic_str 的值就是验证出来的值
code = chaojiying.PostPic(im, 1902)['pic_str']
print(code)
#print chaojiying.PostPic(base64_str, 1902)  #此处为传入 base64代码

lxe8


# 获取登陆后的页面，处理参数

In [25]:
url_login = 'http://so.gushiwen.cn/user/collect.aspx'
data = {
    '__VIEWSTATE': 'ssXMKWPykPSYO7ow01vN1LyMg8mtf0JfNk1JcMbSOxBLxNv4yEsa9OQfY0oJxecqypU0nwMGEqFr88SRxEs7AVO46u2J7A/q8Pq/ApqB8IFZLlmr2/rOXmTk7E7ebh0hWdAsvMNyPZAlac76GzI2NF4rz3I=',
    '__VIEWSTATEGENERATOR': 'C93BE1AE',
    'from': 'http://so.gushiwen.cn/user/collect.aspx',
    'email': '13962970517',
    'pwd': '8291kka',
    'code': code,
    'denglu': '登录'
}

# 用响应对象的响应状态码来判断是否模拟成功
response = requests.post(url_login, headers = headers, data = data)
print(response.status_code)

# 本地持久化存储来判断是否模拟登录成功【这么做不建议】
# page_login = requests.post(url_login, headers = headers, data = data).text
# with open('./login.html', 'w', encoding='utf-8') as fp:
#     fp.write(page_login)
# print('OKKKK')

# 存取详情页面
url_detail = 'https://so.gushiwen.cn/user/collect.aspx'
page_detail = requests.get(url = url_detail, headers = headers).text
with open('./detail.html', 'w', encoding = 'utf-8') as fp:
    fp.write(page_detail)



200
